In [1]:
import pandas as pd
from datetime import datetime
from lxml import html
import requests

In [4]:
def get_injury_report(site):
    """
    Retrieves MLB injury information from the CBS Sports website.
    """
    
    page = requests.get(site)
    tree = html.fromstring(page.content)
    
    # Clean and organize the data
    players, positions, injuries, return_dates = [], [], [], []
    for i in range(1,20):
        player = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[1]/span[2]/span/a/text()")
        [players.append(j) for j in player]

        position = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[2]/text()")
        for j in position:
            positions.append(j.split()[0])

        injury = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[4]/text()")
        for j in injury:
            injuries.append(j.split()[0])

        return_date = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[5]/text()")
        for j in return_date:
            if j.split()[-1] == "season":
                return_dates.append(datetime.strptime("Nov 1 2021", "%b %d %Y"))
            else:
                return_dates.append(datetime.strptime(j.split()[-2]+" "+j.split()[-1]+" 2021","%b %d %Y"))
    
    # Create a dataframe with the organized data
    df = pd.DataFrame({"Name": players, "Pos": positions, "Return": return_dates,
                       "Injury": injuries})

    length = df["Return"] - datetime.today()
    length = [i.round("D") for i in length]
    df["Length"] = length
    df = df[(df["Length"] > "0 days")]
    
    return df

injury_report = get_injury_report("https://www.cbssports.com/mlb/injuries/")
injury_report.describe()

,Length
count,318
mean,31 days 06:43:01.132075471
std,39 days 19:20:52.096145558
min,1 days 00:00:00
25%,3 days 00:00:00
50%,10 days 12:00:00
75%,37 days 06:00:00
max,111 days 00:00:00
